In [45]:
import requests
import json
import time
import pandas as pd
import math

from tqdm import tqdm
from urllib import parse
import pprint as pp


In [12]:
import warnings
warnings.filterwarnings(action='ignore')

MatchID로부터 한타를 발견 및 분석하기 위해 사용할 dataframe
- Champion Kill event관련 dataframe 3개
    - req.json()['info']['frames'][i]['events'] 중 event['type'] == "CHAMPION_KILL" 인 경우
    - frame, 참여자 ID에 상관 없이 모든 Champion Kill event 발생시 수집됨
    
- damageStats 관련 dataframe 1개
    - req.json()['info']['frames'][i]['participantFrames'][j]['damageStats']
    - i = 1, 2, ..., 29 (해당 게임 frame 수 = 플레이 시간)
    - j = 1,2,3, ... , 10 (해당 게임 참여자 ID)
    - event가 아니므로 매 분, 매 참여자마다 수집됨
    

## API access

In [3]:
my_api = "RGAPI-0e13f38a-fe82-4f14-956c-12c3874b2259"
# api는 매일 갱신해주세요

match_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/{}/timeline?api_key=' + my_api

In [39]:
req = requests.get(match_url.format('KR_6513717492')) # 특정 Match id 를 이미 수집하였다고 가정. 
# KR_6513717492 
# req.json()

In [32]:
req.json()['info']['frames'][i]['participantFrames'][j]['damageStats']

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

In [40]:
req.json()['info']['frames'][i]['events']

[{'assistingParticipantIds': [1, 2, 5],
  'bounty': 300,
  'killStreakLength': 0,
  'killerId': 4,
  'position': {'x': 6868, 'y': 9850},
  'shutdownBounty': 0,
  'timestamp': 1681732,
  'type': 'CHAMPION_KILL',
  'victimDamageDealt': [{'basic': False,
    'magicDamage': 0,
    'name': 'Sett',
    'participantId': 1,
    'physicalDamage': 0,
    'spellName': 'settw',
    'spellSlot': 1,
    'trueDamage': 642,
    'type': 'OTHER'},
   {'basic': False,
    'magicDamage': 0,
    'name': 'Sett',
    'participantId': 4,
    'physicalDamage': 0,
    'spellName': 'settw',
    'spellSlot': 1,
    'trueDamage': 642,
    'type': 'OTHER'},
   {'basic': False,
    'magicDamage': 0,
    'name': 'Sett',
    'participantId': 5,
    'physicalDamage': 0,
    'spellName': 'settw',
    'spellSlot': 1,
    'trueDamage': 642,
    'type': 'OTHER'}],
  'victimDamageReceived': [{'basic': False,
    'magicDamage': 208,
    'name': 'Nidalee',
    'participantId': 2,
    'physicalDamage': 0,
    'spellName': '',


# 재료 준비

## Champion Kill Events to dataframe

In [52]:
# Access the relevant data from req.json()
frames = req.json()['info']['frames']

# Initialize empty lists to store the collected data
champion_kill_data = []
victim_damage_dealt_data = []
victim_damage_received_data = []

# Initialize a counter for the kill ID
kill_id_counter = 1

# Iterate over the frames
for i in range(len(frames)):
    events = frames[i]['events']
    # Iterate over the events in each frame
    for event in events:
        if event['type'] == "CHAMPION_KILL": # event type이 CHAMPION_KILL일 때

            try:
                assisting_ids = event['assistingParticipantIds']
            except KeyError:
                assisting_ids = []
                
            champion_kill_data.append({
                'kill_id': kill_id_counter,
                'assistingParticipantIds': assisting_ids,
                #'bounty': event['bounty'],
                #'killStreakLength': event['killStreakLength'],
                'killerId': event['killerId'],
                'position_x': event['position']['x'],
                'position_y': event['position']['y'],
                'timestamp': event['timestamp'],
                'victimId': event['victimId'],
                'minute' : math.floor(event['timestamp']/60000) # 새로 추가함
            })
            kill_id_counter += 1
            
            if 'victimDamageDealt' in event:
                damage_dealt = event['victimDamageDealt']
                for damage in damage_dealt:
                    victim_damage_dealt_data.append({
                        'kill_id': kill_id_counter,
                        'participantId': event['victimId'],
                        'damageType': damage['type'],
                        'damageName': damage['name'],
                        'spellName': damage['spellName'],
                        'spellSlot': damage['spellSlot'],
                        'physicalDamage': damage['physicalDamage'],
                        'magicDamage': damage['magicDamage'],
                        'trueDamage': damage['trueDamage'],
                        'basicAttack': damage['basic']
                    })
            if 'victimDamageReceived' in event:
                damage_received = event['victimDamageReceived']
                for damage in damage_received:
                    victim_damage_received_data.append({
                        'kill_id': kill_id_counter,
                        'participantId': event['victimId'],
                        'damageType': damage['type'],
                        'damageName': damage['name'],
                        'spellName': damage['spellName'],
                        'spellSlot': damage['spellSlot'],
                        'physicalDamage': damage['physicalDamage'],
                        'magicDamage': damage['magicDamage'],
                        'trueDamage': damage['trueDamage'],
                        'basicAttack': damage['basic']
                    })

# Create DataFrames from the collected data
champion_kill_df = pd.DataFrame(champion_kill_data)
victim_damage_dealt_df = pd.DataFrame(victim_damage_dealt_data)
victim_damage_received_df = pd.DataFrame(victim_damage_received_data)

In [53]:
champion_kill_df

,kill_id,assistingParticipantIds,killerId,position_x,position_y,timestamp,victimId,minute
0,1,[1],2,1559,11522,200934,6,3
1,2,[5],4,11020,2308,225621,7,3
2,3,"[7, 10]",9,10525,1439,229728,5,3
3,4,[],4,11458,1698,234399,9,3
4,5,[],4,10996,4074,239791,10,3
...,...,...,...,...,...,...,...,...
62,63,"[1, 2, 5]",4,6868,9850,1681732,8,28
63,64,"[3, 4, 5]",2,8217,7717,1689636,6,28
64,65,"[1, 3, 5]",4,10592,10975,1698633,10,28
65,66,"[1, 3]",2,10016,12081,1699195,9,28


In [36]:
victim_damage_dealt_df

,kill_id,participantId,damageType,damageName,spellName,spellSlot,physicalDamage,magicDamage,trueDamage,basicAttack
0,2,6,OTHER,Fiora,fiorabasicattack2,65,262,0,0,True
1,2,6,OTHER,Fiora,fioraq,0,118,0,0,False
2,2,6,OTHER,Fiora,fiorae,2,146,0,0,False
3,2,6,OTHER,Fiora,fiorabasicattack,64,65,0,0,True
4,2,6,OTHER,Fiora,fioraw,1,0,90,0,False
...,...,...,...,...,...,...,...,...,...,...
346,67,9,OTHER,Lucian,,-1,44,0,0,False
347,67,9,OTHER,Lucian,lucianpassiveattack,0,266,0,0,False
348,67,9,OTHER,Lucian,lucianq,0,133,0,0,False
349,67,9,OTHER,Lucian,lucianw,1,0,46,0,False


In [37]:
victim_damage_received_df

,kill_id,participantId,damageType,damageName,spellName,spellSlot,physicalDamage,magicDamage,trueDamage,basicAttack
0,2,6,OTHER,Ornn,ornnw,1,0,173,0,False
1,2,6,OTHER,Ornn,ornne,2,56,0,0,False
2,2,6,OTHER,Ornn,ornnbasicattack3,66,50,0,0,True
3,2,6,OTHER,Ornn,ornnbasicattack,64,50,0,0,True
4,2,6,OTHER,Ornn,,-1,0,49,0,False
...,...,...,...,...,...,...,...,...,...,...
860,67,9,OTHER,Ornn,ornnq,0,131,0,0,False
861,67,9,OTHER,Nidalee,,-1,0,184,19,False
862,67,9,OTHER,Nidalee,pounce,1,0,337,0,False
863,67,9,OTHER,Nidalee,takedown,0,0,786,0,False


추가로 해야 할 것
- timestamp 열 외에 minute 열 추가(언제인지 와닿지 않음)
- 킬 수가 5분 내, 5건 이상 발생하면 한타 발생 가정 
- 한타 definition 의 output은?
    - 한타 발생시 frame
    - damage stat dataframe
    - kill dataframe

## Damage Stat dataframe

In [33]:
url4='https://asia.api.riotgames.com/lol/match/v5/matches/'+'KR_6513717492'+'/timeline?api_key='+my_api
r = requests.get(url4)
c = r.json() 

In [48]:
def make_damagestat_row(participantlist,timestamp):
  final_frame=pd.DataFrame()
  for i in participantlist:
    value_1=pd.DataFrame(c['info']['frames'][timestamp]['participantFrames'][i]['damageStats'].values()).T
    value_1.columns=c['info']['frames'][timestamp]['participantFrames'][i]['damageStats'].keys()
    value_2=pd.DataFrame(c['info']['frames'][timestamp]['participantFrames'][i]['position'].values()).T
    value_2.columns=c['info']['frames'][timestamp]['participantFrames'][i]['position'].keys()
    total_value=pd.concat([value_1,value_2],axis=1)
    total_value['participantId']=c['info']['frames'][timestamp]['participantFrames'][i]['participantId']
    total_value['minutes']=timestamp
    final_frame=pd.concat([final_frame,total_value])
  return final_frame #데미지 스탯, 포지션, 타임스탬프, 참여자 아이디 열 생성

In [49]:
timestamp=len(c['info']['frames'])
partition_list=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [50]:
damage_df=pd.DataFrame()
for i in range(timestamp):
  participant=make_damagestat_row(partition_list,i)
  damage_df=pd.concat([damage_df,participant])

damage_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 0
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   magicDamageDone                300 non-null    int64
 1   magicDamageDoneToChampions     300 non-null    int64
 2   magicDamageTaken               300 non-null    int64
 3   physicalDamageDone             300 non-null    int64
 4   physicalDamageDoneToChampions  300 non-null    int64
 5   physicalDamageTaken            300 non-null    int64
 6   totalDamageDone                300 non-null    int64
 7   totalDamageDoneToChampions     300 non-null    int64
 8   totalDamageTaken               300 non-null    int64
 9   trueDamageDone                 300 non-null    int64
 10  trueDamageDoneToChampions      300 non-null    int64
 11  trueDamageTaken                300 non-null    int64
 12  x                              300 non-null    int64
 13  y                     

In [51]:
damage_df

,magicDamageDone,magicDamageDoneToChampions,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,x,y,participantId,minutes
0,0,0,0,0,0,0,0,0,0,0,0,0,554,581,1,0
0,0,0,0,0,0,0,0,0,0,0,0,0,593,464,2,0
0,0,0,0,0,0,0,0,0,0,0,0,0,557,345,3,0
0,0,0,0,0,0,0,0,0,0,0,0,0,458,271,4,0
0,0,0,0,0,0,0,0,0,0,0,0,0,335,269,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3162,1182,16092,130111,20435,22031,158831,35865,39655,25557,14247,1531,8217,7717,6,29
0,151007,23827,10035,17442,1155,14318,197502,26596,25223,29052,1614,869,11052,13380,7,29
0,105,29,10553,87875,8786,17773,92504,11629,29324,4523,2813,997,6868,9850,8,29
0,14431,2505,7729,123405,12589,9430,138172,15431,17234,336,336,75,10016,12081,9,29


- 0분에 10명치, 1분에 10명치 , ... , 29분에 10명치 damage status 이므로 총 300개의 row가 발생함
- **timestamp열은 밀리초단위가 아니라 frame 번호를 의미함. -> minute로이름 변경**
    - kill event, 다른 event dataframes 와의 통일성을 위해 밀리초단위열도 필요할지도


추가로 해야 할 것 : 
- 10명분 totalDamageDoneToChampions을 합해서 분당 전체 주고받은 데미지를 구하기
- 분당 데미지 증가율이 유의미하게 높은 경우(관전하며 판단한 실제 한타 발생시점을 기준으로) 한타 발생 가정
    - 일단 kill event 발생 기준으로 한타 감지하기로 함 (시간나면 해보기) 


## detect 한타

In [83]:
def detect_high_kill_intervals(champion_kill_df):
    # Sort the DataFrame by 'minute' column in ascending order
    champion_kill_df_sorted = champion_kill_df.sort_values('minute')
    
    # Initialize variables
    intervals = []
    max_kills = 6  # Maximum number of kills within an interval
    interval_length = 2  # Length of each interval in minutes
    
    # Iterate over the minutes in the range from 0 to (interval_length - 1) minutes before the last minute
    for start_minute in range(champion_kill_df_sorted['minute'].min(), champion_kill_df_sorted['minute'].max() - interval_length + 1):
        end_minute = start_minute + interval_length
        
        # Get the subset of DataFrame within the current interval
        interval_df = champion_kill_df_sorted[(champion_kill_df_sorted['minute'] >= start_minute) & (champion_kill_df_sorted['minute'] < end_minute)]
        
        # Count the number of kills within the interval
        kill_count = len(interval_df)
        
        # Check if the kill count is greater than the maximum
        if kill_count > max_kills:
            intervals.append(start_minute)
    
    return intervals



In [84]:
detect_high_kill_intervals(champion_kill_df)

[3, 15, 18, 19, 21, 22, 24]

너무많아서 기준 강화함. 여전히 많음 .. 